In [1]:
from ast import Param

from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition

import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

import plotly.express as px
import plotly.graph_objects as go
from pyomo.common.config import PositiveInt
from pyomo.core import ConcreteModel

In [23]:
os.environ["NEOS_EMAIL"] = "antonsivko480@gmail.com"

In [3]:
# solver = SolverFactory(
#     "cbc",
#     executable=r"C:\Users\Antonio\Desktop\New folder\bin\cbc.exe"
# )
# print("Is solver available? ->", solver.available())
# print("Is solver executable? ->", solver.executable())


# Base Paramethers & Data

## Base Params (st70)

In [4]:
n_value = 70
s_value = 1
k_value = 10

## # st70 Coordinates


In [5]:
xcoord_list = [
    64, 80, 69, 72, 48, 58, 81, 79, 30, 42,
    7, 29, 78, 64, 95, 57, 40, 68, 92, 62,
    28, 76, 67, 93, 6, 87, 30, 77, 78, 55,
    82, 73, 20, 27, 95, 67, 48, 75, 8, 20,
    54, 63, 44, 52, 12, 25, 58, 5, 90, 41,
    25, 37, 56, 10, 98, 16, 89, 48, 81, 29,
    17, 5, 79, 9, 17, 74, 10, 48, 83, 84,
]

ycoord_list = [
    96, 39, 23, 42, 67, 43, 34, 17, 23, 67,
    76, 51, 92, 8, 57, 91, 35, 40, 34, 1,
    43, 73, 88, 54, 8, 18, 9, 13, 94, 3,
    88, 28, 55, 43, 86, 99, 83, 81, 19, 18,
    38, 36, 33, 18, 13, 5, 85, 67, 9, 76,
    76, 64, 63, 55, 7, 74, 60, 82, 76, 60,
    22, 45, 70, 100, 82, 67, 68, 19, 86, 94,
]

xcoord_data = {i + 1: xcoord_list[i] for i in range(n_value)}
ycoord_data = {i + 1: ycoord_list[i] for i in range(n_value)}

## Distance


In [6]:
d_data = {}

for i in range(1, n_value + 1):
    for j in range(1, n_value + 1):
        if i != j:
            dx = xcoord_data[i] - xcoord_data[j]
            dy = ycoord_data[i] - ycoord_data[j]
            dist = math.hypot(dx, dy)
            d_data[(i, j)] = int(round(dist, 2))

# print("d_data:", d_data)

# Pyomo Model

In [7]:
model = ConcreteModel()

## Model Parameters


In [8]:
model.n = Param(initialize=n_value, within=PositiveIntegers)
model.s = Param(initialize=s_value, within=PositiveIntegers)
model.k = Param(initialize=k_value, within=PositiveIntegers)

## Nodes & Arcs


In [9]:
model.NODES = RangeSet(1, model.n)

def arcs_init(model):
    return [(i, j) for i in model.NODES for j in model.NODES if i != j]

model.ARCS = Set(dimen=2, initialize=arcs_init)

## x_coord, y_coord, d


In [10]:
model.x_coord = Param(model.NODES, initialize=xcoord_data)
model.y_coord = Param(model.NODES, initialize=ycoord_data)

model.d = Param(model.ARCS, within=NonNegativeReals, initialize=d_data)

## x, y, z, u variables


In [11]:
model.x = Var(model.ARCS, domain=Binary)
model.y = Var(model.NODES, domain=Binary)
model.z = Var(model.ARCS, domain=NonNegativeReals)

def u_bounds(m, i):
    return (1, m.k)

model.u = Var(model.NODES, domain=Integers, bounds=u_bounds)

## Target Function


In [12]:
def dk_min_rule(m):
    return sum(m.d[i, j] * m.x[i, j] for (i, j) in m.ARCS)

model.dk_min = Objective(rule=dk_min_rule, sense=minimize)

## con2 - con4


In [13]:
def con2_rule(m, i):
    if i == m.s:
        rhs = 1
    else:
        rhs = m.y[i]
    return sum(m.x[i, j] for j in m.NODES if (i, j) in m.ARCS) == rhs

model.con2 = Constraint(model.NODES, rule=con2_rule)

def con3_rule(m, i):
    if i == m.s:
        rhs = 1
    else:
        rhs = m.y[i]
    return sum(m.x[j, i] for j in m.NODES if (j, i) in m.ARCS) == rhs

model.con3 = Constraint(model.NODES, rule=con3_rule)

def con4_rule(m):
    return sum(m.y[i] for i in m.NODES if i != m.s) == m.k

model.con4 = Constraint(rule=con4_rule)

## Flow-based Part (con5 - con7)

In [14]:
def con5_rule(m, i, j):
    return m.z[i, j] - m.k * m.x[i, j] <= 0

model.con5 = Constraint(model.ARCS, rule=con5_rule)

def con6_1_rule(m):
    # Maybe a problem later
    s = value(m.s)
    return sum(m.z[m.s, j] for j in m.NODES if (s, j) in m.ARCS) == m.k

model.con6_1 = Constraint(rule=con6_1_rule)

def con6_2_rule(m):
    # Maybe a problem later
    s = value(m.s)
    return sum(m.z[j, m.s] for j in m.NODES if (j, s) in m.ARCS) == 0

model.con6_2 = Constraint(rule=con6_2_rule)

def con7_rule(m, i):
    if i == m.s:
        return Constraint.Skip
    outgoing = sum(m.z[i, j] for j in m.NODES if (i, j) in m.ARCS)
    incoming = sum(m.z[j, i] for j in m.NODES if (j, i) in m.ARCS)
    return outgoing - incoming == -m.y[i]

model.con7 = Constraint(model.NODES, rule=con7_rule)

## MTZ Part (con15)

In [15]:
def con15_rule(m, i, j):
    if i == m.s or j == m.s:
        return Constraint.Skip
    return m.u[i] - m.u[j] + m.k * m.x[i, j] <= m.k - 1

model.con15 = Constraint(model.ARCS, rule=con15_rule)

# Solver Part

In [16]:
# Connect NEOS Server

neos = SolverManagerFactory("neos")

In [17]:
# Problem 1

print("Problem 1:")

# con5-con7 is active
model.con5.activate()
model.con6_1.activate()
model.con6_2.activate()
model.con7.activate()

# con15 deactivate
model.con15.deactivate()

results1 = neos.solve(model, opt="cplex", tee=False)

if (results1.solver.status == SolverStatus.ok and
    results1.solver.termination_condition in [TerminationCondition.optimal,
                                              TerminationCondition.feasible]):

    t1 = getattr(results1.solver, "time", None)
    if t1 is not None:
        print("Solve time (problem1):", t1)

    print("dk_min (problem1) =", value(model.dk_min))

    print("Display x (problem1):")
    for i in model.NODES:
        for j in model.NODES:
            if (i, j) in model.ARCS:
                x_val = value(model.x[i, j])
                if x_val >= 0.5:
                    print(f"{i} -> {j} [{x_val:.2f}]")

else:
    print("The solution for problem 2 was not found or is not optimal — see the status above.")


Problem 1:
dk_min (problem1) = 71.0
Display x (problem1):
1 -> 23 [1.00]
13 -> 29 [1.00]
22 -> 63 [1.00]
23 -> 38 [1.00]
29 -> 36 [1.00]
31 -> 13 [1.00]
36 -> 1 [1.00]
38 -> 22 [1.00]
59 -> 69 [1.00]
63 -> 59 [1.00]
69 -> 31 [1.00]


In [30]:
# Problem 2

print("\nProblem 2:")

# con5-con7 is inactive
model.con5.deactivate()
model.con6_1.deactivate()
model.con6_2.deactivate()
model.con7.deactivate()

# con15 is active
model.con15.activate()

results2 = neos.solve(model, opt="cplex", tee=False)

if (results2.solver.status == SolverStatus.ok and
    results2.solver.termination_condition in [TerminationCondition.optimal,
                                              TerminationCondition.feasible]):

    print("dk_min (problem2) =", value(model.dk_min))

    print("Display x (problem2):")
    for i in model.NODES:
        for j in model.NODES:
            if (i, j) in model.ARCS:
                x_val = value(model.x[i, j])
                if x_val is not None and x_val >= 0.5:
                    print(f"{i} -> {j} [{x_val:.2f}]")

    print("Display u (problem 2):")

    for i in model.NODES:
        yi = model.y[i].value
        if yi is not None and yi > 0.5:
            print(f"Node: {i}\tSequence Number:{int(model.u[i].value)}")

    print("Display sorted u (problem 2):")

    selected = []
    for i in model.NODES:
        yi = model.y[i].value
        if yi is None:
            continue
        if yi > 0.5:
            u_val = model.u[i].value
            if u_val is not None:
                selected.append((i, u_val))

    selected.sort(key=lambda t: t[1])

    for i, u in selected:
        out = int(u) if float(u).is_integer() else u
        print(f"Node: {i}\tSequence Number:{out}")

else:
    print("The solution for problem 2 was not found or is not optimal — see the status above.")



Problem 2:
dk_min (problem2) = 71.0
Display x (problem2):
1 -> 23 [1.00]
13 -> 29 [1.00]
22 -> 63 [1.00]
23 -> 38 [1.00]
29 -> 36 [1.00]
31 -> 13 [1.00]
36 -> 1 [1.00]
38 -> 22 [1.00]
59 -> 69 [1.00]
63 -> 59 [1.00]
69 -> 31 [1.00]
Display u (problem 2):
Node: 13	Sequence Number:8
Node: 22	Sequence Number:3
Node: 23	Sequence Number:1
Node: 29	Sequence Number:9
Node: 31	Sequence Number:7
Node: 36	Sequence Number:10
Node: 38	Sequence Number:2
Node: 59	Sequence Number:5
Node: 63	Sequence Number:4
Node: 69	Sequence Number:6
Display sorted u (problem 2):
Node: 23	Sequence Number:1
Node: 38	Sequence Number:2
Node: 22	Sequence Number:3
Node: 63	Sequence Number:4
Node: 59	Sequence Number:5
Node: 69	Sequence Number:6
Node: 31	Sequence Number:7
Node: 13	Sequence Number:8
Node: 29	Sequence Number:9
Node: 36	Sequence Number:10
